In [18]:
%%file myConv.py

import torch
import numpy as np
def myConv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
  def Svertca(matrix):

    #добавление отступов и padding в входной матрице
    dil = stride*2
    result_matrix = []
    for matr in matrix:
      zero_tensor = np.zeros(((matr.shape[0]-1)*dil+1, (matr.shape[1]-1)*dil+1))
      for a in range (0, zero_tensor.shape[0], dil):
        for b in range (0, zero_tensor.shape[1], dil):
          zero_tensor[a][b] = matr[a//dil][b//dil]

      pad_matr = np.pad(zero_tensor, pad_width=dil, mode='constant')
      result_matrix.append(pad_matr)
    matrix = torch.tensor(result_matrix)

    #groups exeption
    if (in_channels%groups != 0) or (out_channels%groups!=0):
      raise Exception('in_channels and out_channels must be divisible by groups')

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad2d(padding)
      matrix = pad(matrix)

    #генерация ядра
    if type(kernel_size) == tuple:
      filter_for_transpose = np.array(torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1]))
    if type(kernel_size) == int:
      filter_for_transpose = np.array(torch.rand(out_channels, in_channels//groups, kernel_size, kernel_size))

    # for j in range(out_channels):
    #   filter_in = []
    #   for i in (filter_for_transpose[j]):
    #     filter_in.append(np.flip(i))
    #   filter.append(filter_in)
    # filter = torch.tensor(filter)


    #инвертирование ядра
    filter = []
    for j in range(in_channels//groups):
      filter_in = []
      for i in range(out_channels):
        filter_in.append(np.flip(filter_for_transpose[i][j]))
      filter.append(filter_in)
    filter = torch.tensor(filter)
    filter = filter.reshape(out_channels, in_channels//groups, kernel_size, kernel_size)

    spisok = []
    for l in range(out_channels):
      feature_map = np.array([]) #генерация пустой feature-map
      for i in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride): #(filter.size - 1)*dilation + 1 при delation
        for j in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
          summa = 0
          for c in range (in_channels//groups): #groups
            if groups>1:
              val = matrix[l*(in_channels//groups)+c-1][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            else:
              val = matrix[c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            mini_sum = (val*filter[l][c]).sum()
            summa = summa + mini_sum
          feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias

      spisok.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1))

    filter_for_transpose = filter_for_transpose.reshape(in_channels, out_channels//groups, kernel_size, kernel_size)
    return np.array(spisok), np.array(filter_for_transpose), np.array(bias_val)

  return Svertca

Overwriting myConv.py


In [19]:
%%file conftest.py

from pytest import fixture
import requests

@fixture
def session():
  s = requests.Session()
  s.headers.update({"User-Agent": "pytest requests"})
  s.verify = True
  yield s
  s.close()

Writing conftest.py


In [22]:
%%file test_http.py

import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor1 = torch.rand(3, 5, 6)
tensor2 = torch.rand(2, 28, 28)
tensor3 = torch.rand(7, 10, 10)


def test_1(session):
  myFunction = myConv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, bias=True,)
  result, kernel, bias_val = myFunction(tensor1)
  torchFunction = torch.nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=3, stride=2, bias=True,)
  torchFunction.weight.data = torch.tensor(kernel)
  torchFunction.bias.data = torch.tensor(bias_val)
  myResult = str(np.round(result, 2))
  torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
  assert torchResult == myResult


def test_2(session):
  myFunction2 = myConv2d(in_channels=2, out_channels=2, kernel_size=3, stride=1, bias=True)
  result2, kernel2, bias_val2 = myFunction2(tensor2)
  torchFunction2 = torch.nn.ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=3, stride=2, bias=True)
  torchFunction2.weight.data = torch.tensor(kernel2)
  torchFunction2.bias.data = torch.tensor(bias_val2)
  myResult = str(np.round(result2, 2))
  torchResult = str(np.round(np.array(torchFunction2(tensor2).data), 2))
  assert torchResult == myResult


def test_3(session):
  myFunction3 = myConv2d(in_channels=7, out_channels=1, kernel_size=1, stride=1, bias=True)
  result3, kernel3, bias_val3 = myFunction3(tensor3)
  torchFunction3 = torch.nn.Conv2d(in_channels=7, out_channels=1, kernel_size=1, stride=2, bias=True)
  torchFunction3.weight.data = torch.tensor(kernel3)
  torchFunction3.bias.data = torch.tensor(bias_val3)
  myResult = str(np.round(result3, 2))
  torchResult = str(np.round(np.array(torchFunction3(tensor3).data), 2))
  assert torchResult == myResult


Overwriting test_http.py


In [23]:
!pytest

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.3, pluggy-1.3.0
rootdir: /content
plugins: anyio-3.7.1
collected 3 items                                                                                  

test_http.py ..F                                                                             [100%]

============================================= FAILURES =============================================
______________________________________________ test_3 ______________________________________________

session = <requests.sessions.Session object at 0x7e5caa369ea0>

    def test_3(session):
      myFunction3 = myConv2d(in_channels=7, out_channels=1, kernel_size=1, stride=1, bias=True)
      result3, kernel3, bias_val3 = myFunction3(tensor3)
      torchFunction3 = torch.nn.Conv2d(in_channels=7, out_channels=1, kernel_size=1, stride=2, bias=True)
      torchFunction3.weight.data = torch.tenso

In [4]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor1 = torch.rand(3, 4, 5)

myFunction = myConv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, bias=True)
result, kernel, bias_val = myFunction(tensor1)

torchFunction = torch.nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=3, stride=2, bias=True)
torchFunction.weight.data = torch.tensor(kernel)
torchFunction.bias.data = torch.tensor(bias_val)

print(result)
print('===================================')
print(np.array(torchFunction(tensor1).data))

[[[1.42308282 1.61508284 2.78809731 1.32917273 2.42411014 1.21289843
   2.50870837 1.50891402 2.62144129 1.37332491 2.05843107]
  [1.84924548 1.71099724 2.65230698 1.58864066 2.20387612 1.54567683
   2.45503471 1.76435583 2.71942506 1.72519637 1.47186505]
  [2.09030575 2.46122771 4.05112998 2.25575669 4.30605885 2.56926876
   4.36596325 2.93307161 4.70334626 2.17589075 2.46260732]
  [1.62510587 1.27639589 2.35298792 1.40064638 2.45572808 1.29418007
   3.25056745 1.70504725 2.42158608 1.21292206 0.95754677]
  [1.76985943 2.03830343 3.59661632 1.98527541 3.23354919 1.93319525
   3.57839041 2.1045406  3.90754496 2.11927045 2.4895968 ]
  [1.70626393 1.64285423 2.37493505 1.28550215 2.2488487  1.33643743
   2.09770638 1.36063984 2.14267824 1.5171639  1.52508163]
  [2.01822003 2.4378459  3.71404788 2.40032417 3.95795428 1.81908533
   3.93374576 1.8357828  2.99290338 1.86059286 1.66990414]
  [1.67215526 1.17830965 3.04978746 1.84119021 2.50305326 1.53338492
   1.5296587  1.06970252 1.50766881

In [6]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor2 = torch.rand(2, 3, 3)

myFunction2 = myConv2d(in_channels=2, out_channels=2, kernel_size=3, stride=1, bias=True)
result, kernel, bias_val = myFunction2(tensor2)

torchFunction2 = torch.nn.ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=3, stride=2, bias=True)
torchFunction2.weight.data = torch.tensor(kernel)
torchFunction2.bias.data = torch.tensor(bias_val)

print(result)
print('===================================')
print(np.array(torchFunction2(tensor2).data))

[[[0.90484725 0.9570017  1.79164029 1.01770131 1.62797559 0.85689489
   0.93124193]
  [0.16762969 0.62787    0.89783409 0.68252064 1.10545438 0.69183299
   0.63443142]
  [1.17115124 1.53470132 1.92193765 1.07468154 1.77625727 0.97373842
   0.8863548 ]
  [0.25301129 0.5897094  0.84888724 0.32979236 0.35813384 0.31395195
   0.28738557]
  [1.30207496 1.63233611 2.50362792 0.82324604 1.48076794 0.8312828
   0.96987135]
  [0.54980106 0.91303648 0.9365139  0.49079355 0.93377978 0.59022426
   0.41679823]
  [0.83195122 0.73407158 1.03031397 0.52694407 0.90745049 0.36313886
   0.41013703]]

 [[1.64133673 1.4143868  2.11182021 1.46428611 2.09836106 1.46952818
   1.4461865 ]
  [1.78365864 1.65559184 2.30588766 1.70978965 2.12900883 1.58938217
   1.49151934]
  [2.25205429 1.52610248 2.96560109 1.2996348  2.72348504 1.29852409
   1.81382811]
  [1.58135745 1.5278804  1.5390766  1.10978888 1.516043   1.17293006
   1.13415598]
  [2.38962852 1.81517017 2.94184242 1.37152232 2.35609651 1.44864205
   1.7

In [15]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor3 = torch.rand(2, 3, 3)

myFunction3 = myConv2d(in_channels=2, out_channels=1, kernel_size=3, stride=1, bias=True)
result, kernel, bias_val = myFunction3(tensor3)

torchFunction3 = torch.nn.ConvTranspose2d(in_channels=2, out_channels=1, kernel_size=3, stride=2, bias=True)
torchFunction3.weight.data = torch.tensor(kernel)
torchFunction3.bias.data = torch.tensor(bias_val)

print(result)
print('===================================')
print(np.array(torchFunction3(tensor3).data))

[[[1.77486066 1.5997003  2.17547702 1.28030976 1.59707308 1.14332847
   1.2236248 ]
  [2.33633608 1.61734453 2.53299377 1.28300422 1.78958147 1.04448222
   1.26291928]
  [1.92429709 1.45294706 3.02106907 1.65617497 2.795411   1.47275368
   1.86318928]
  [1.43426511 1.35718038 2.37475281 1.62389851 2.69353382 1.32524031
   1.67333688]
  [1.62454552 1.42276382 2.88779818 1.75537672 3.11957554 1.4982327
   2.1258816 ]
  [1.57223396 1.06001483 2.59865213 1.76951787 2.32880066 1.17877005
   1.56505285]
  [1.20177318 1.00577717 2.05950101 1.30202484 1.83488571 1.0561818
   1.49363257]]]
[[[1.7748606 1.5997003 2.175477  1.2803098 1.5970731 1.1433284 1.2236248]
  [2.3363361 1.6173446 2.5329938 1.2830043 1.7895815 1.0444822 1.2629193]
  [1.9242971 1.4529471 3.021069  1.6561749 2.795411  1.4727538 1.8631892]
  [1.4342651 1.3571804 2.3747528 1.6238985 2.693534  1.3252404 1.6733369]
  [1.6245456 1.4227638 2.887798  1.7553768 3.1195755 1.4982326 2.1258817]
  [1.5722339 1.0600148 2.5986521 1.7695179